In [1]:
from multiprocessing import freeze_support

import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, Sampler
from torchvision import datasets
from torchvision import models
from torchvision.transforms import transforms
from torch.optim import Adam


import matplotlib.pyplot as plt
import numpy as np

from pathlib import Path

In [2]:
# Hyperparameters.
num_epochs = 20
num_classes = 5
batch_size = 100
learning_rate = 0.001
num_of_workers = 5

In [3]:
DATA_PATH_TRAIN = Path('./flowers')
DATA_PATH_TEST = Path('./unlabel-flowers')
MODEL_STORE_PATH = Path('./')

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
trans = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Resize(32),
    transforms.CenterCrop(32),
    transforms.ToTensor(),
    #transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
    ])

In [6]:
# Flowers dataset.
train_dataset = datasets.ImageFolder(root=DATA_PATH_TRAIN, transform=trans)
test_dataset = datasets.ImageFolder(root=DATA_PATH_TEST, transform=trans)
# Create custom random sampler class to iter over dataloader.
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_of_workers)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_of_workers)

In [7]:

# CNN we are going to implement.
class Unit(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Unit, self).__init__()

        self.conv = nn.Conv2d(in_channels=in_channels, kernel_size=3, out_channels=out_channels, stride=1, padding=1)
        self.bn = nn.BatchNorm2d(num_features=out_channels)
        self.relu = nn.ReLU()

    def forward(self, input):
        output = self.conv(input)
        output = self.bn(output)
        output = self.relu(output)

        return output

In [8]:
class CNNet(nn.Module):
    def __init__(self, num_class):
        super(CNNet, self).__init__()

        # Create 14 layers of the unit with max pooling in between
        self.unit1 = Unit(in_channels=3, out_channels=32)
        self.unit2 = Unit(in_channels=32, out_channels=32)
        self.unit3 = Unit(in_channels=32, out_channels=32)

        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.unit4 = Unit(in_channels=32, out_channels=64)
        self.unit5 = Unit(in_channels=64, out_channels=64)
        self.unit6 = Unit(in_channels=64, out_channels=64)
        self.unit7 = Unit(in_channels=64, out_channels=64)

        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.unit8 = Unit(in_channels=64, out_channels=128)
        self.unit9 = Unit(in_channels=128, out_channels=128)
        self.unit10 = Unit(in_channels=128, out_channels=128)
        self.unit11 = Unit(in_channels=128, out_channels=128)

        self.pool3 = nn.MaxPool2d(kernel_size=2)

        self.unit12 = Unit(in_channels=128, out_channels=128)
        self.unit13 = Unit(in_channels=128, out_channels=128)
        self.unit14 = Unit(in_channels=128, out_channels=128)

        self.avgpool = nn.AvgPool2d(kernel_size=4)

        # Add all the units into the Sequential layer in exact order
        self.net = nn.Sequential(self.unit1, self.unit2, self.unit3, self.pool1, self.unit4, self.unit5, self.unit6
                                 , self.unit7, self.pool2, self.unit8, self.unit9, self.unit10, self.unit11, self.pool3,
                                 self.unit12, self.unit13, self.unit14, self.avgpool)

        self.fc = nn.Linear(in_features=128, out_features=num_class)

    def forward(self, input):
        output = self.net(input)
        output = output.view(-1, 128)
        output = self.fc(output)
        return output







In [9]:
# Create model, optimizer and loss function
#model = CNNet(num_classes)

model = models.resnet101(pretrained=True)
fc_features = model.fc.in_features
model.fc = nn.Linear(fc_features, 5)

# if cuda is available, move the model to the GPU

model.cuda()
# Define the optimizer and loss function
optimizer = Adam(model.parameters(), lr=0.0001, weight_decay=0.0001)
loss_fn = nn.CrossEntropyLoss()

In [10]:
def save_models(epoch):
    torch.save(model.state_dict(), f"{epoch}.model")
    print("Checkpoint saved")

In [11]:
def test():
    model.eval()
    test_acc = 0.0
    count=0
    for i, (images, labels) in enumerate(test_loader):

        images = images.to(device)
        labels = labels.to(device)

        # Predict classes using images from the test set
        outputs = model(images)
        _, prediction = torch.max(outputs.data, 1)

        test_acc += torch.sum(prediction == labels.data).float()

    # Compute the average acc and loss over all 10000 test images
    test_acc = test_acc / 175 * 100
    return test_acc

In [12]:
def train(num_epoch):
    best_acc = 0.0

    for epoch in range(num_epoch):
        model.train()
        train_acc = 0.0
        train_loss = 0.0
        for i, (images, labels) in enumerate(train_loader):
            # Move images and labels to gpu if available
            

            images = images.to(device)
            labels = labels.to(device)

            # Clear all accumulated gradients
            optimizer.zero_grad()
            # Predict classes using images from the test set
            outputs = model(images)
            # Compute the loss based on the predictions and actual labels
            loss = loss_fn(outputs, labels)
            # Backpropagate the loss
            loss.backward()

            # Adjust parameters according to the computed gradients
            optimizer.step()

            #train_loss += loss.cpu().data[0] * images.size(0)
            train_loss += loss.item()
            _, prediction = torch.max(outputs.data, 1)
           

            train_acc += torch.sum(prediction == labels.data).float()

        # Call the learning rate adjustment function
        #adjust_learning_rate(epoch)

        # Compute the average acc and loss over all 50000 training images
        train_acc = train_acc / 4153 * 100
        train_loss = train_loss / 8306

        # Evaluate on the test set
        test_acc = test()

        # Save the model if the test acc is greater than our current best
        if train_acc > best_acc:
            save_models(epoch)
            best_acc = train_acc

        # Print the metrics
        print(f"Epoch {epoch + 1}, Train Accuracy: {train_acc} , TrainLoss: {train_loss} , Test Accuracy: {test_acc}")

In [14]:
if __name__ == '__main__':
    freeze_support()
    train(num_epochs)

Checkpoint saved
Epoch 1, Train Accuracy: 79.31615447998047 , TrainLoss: 0.00287138053000533 , Test Accuracy: 23.428571701049805
Checkpoint saved
Epoch 2, Train Accuracy: 87.38261413574219 , TrainLoss: 0.0018110046387005794 , Test Accuracy: 23.428571701049805
Checkpoint saved
Epoch 3, Train Accuracy: 91.6927490234375 , TrainLoss: 0.0012727520854005807 , Test Accuracy: 21.14285659790039
Checkpoint saved
Epoch 4, Train Accuracy: 93.95616912841797 , TrainLoss: 0.0009142939210325558 , Test Accuracy: 20.571428298950195
Checkpoint saved
Epoch 5, Train Accuracy: 95.42499542236328 , TrainLoss: 0.0007171274216425382 , Test Accuracy: 18.857141494750977
Checkpoint saved
Epoch 6, Train Accuracy: 96.46038818359375 , TrainLoss: 0.0005857957517622811 , Test Accuracy: 20.571428298950195
Checkpoint saved
Epoch 7, Train Accuracy: 96.62893676757812 , TrainLoss: 0.0005289374200581012 , Test Accuracy: 22.285715103149414
Checkpoint saved
Epoch 8, Train Accuracy: 97.56802368164062 , TrainLoss: 0.000463454811